In [13]:
import polars as pl
import spacy
import torch
from collections import namedtuple
from preprocess import load_data
from transformers import  AutoTokenizer
import joblib



In [14]:
# args
model_directory = "C:/Users/prl90/PycharmProjects/kaggle_lstm/bert_tiny/"
stride = 20
max_length  = 450
test_path = 'C:/Users/prl90/PycharmProjects/kaggle_lstm/data/train.json'


In [15]:

def load_data(file_path, train=True):
    df = pl.read_json(file_path).lazy()
    reshaped_df = (
        df
        .with_columns(
            token_keys=pl.col('tokens')
            .map_elements(lambda f: [i for i in range(len(f))]),
        )
        .explode(['tokens', 'trailing_whitespace', 'token_keys'])
        .with_columns(
            token_len=pl.col('tokens').str.len_bytes(),
        )
        .with_columns(
            token_len=pl.struct(['token_keys', 'token_len'])
            .map_elements(lambda f: [f['token_keys']] * f['token_len'])
        )
        .with_columns(
            token_len=pl.when(pl.col('trailing_whitespace') == True)
            .then(pl.col('token_len').list.concat(-1))
            .otherwise(pl.col('token_len')),
            tokens_rec=pl.when(pl.col('trailing_whitespace') == True)
            .then(pl.col('tokens').add(' '))
            .otherwise(pl.col('tokens'))
        )
        .group_by('document', maintain_order=True)
        .agg(
            tokens=pl.col('tokens'),
            token_key=pl.col('token_len').flatten(),
            tokens_rec=pl.col('tokens_rec')
        )
        .with_columns(
            token_rec=pl.col('tokens_rec')
            .list.join(separator='')
        )
    )
    if train:
        temp = ((reshaped_df
                 .join(df.select(['document', 'labels'])
                       , on='document')
                 )
        )
    else:
        temp = reshaped_df
    return temp

In [16]:
 temp = load_data(test_path, train = True)
    

In [17]:

def tokenize( r, model_directory=model_directory,stride = stride, max_length = max_length ):
    txt = r['token_rec']
    tokenizer = AutoTokenizer.from_pretrained(model_directory)
    
    inputs = tokenizer(txt , 
                       return_tensors="pt", 
                       padding=True, 
                       truncation=True, 
                       return_offsets_mapping=True, 
                       return_overflowing_tokens = True,
                       stride = stride,
                       add_special_tokens=False,
                       max_length= max_length)
    
    offset = inputs['offset_mapping'].tolist()
    attention_mask = inputs['attention_mask'].tolist()
    input_ids = inputs['input_ids'].tolist()
    
    
    txt_= []
    attention_mask_=[]
    input_ids_=[]
    offset_=[]
    

    for i, j in enumerate(offset):
        txt_.append(tokenizer.convert_ids_to_tokens(inputs["input_ids"][i].squeeze()))
        attention_mask_.append(attention_mask[i])
        input_ids_.append(input_ids[i])
        offset_.append(j)
    return {'txt':txt_, 'attention_mask': attention_mask_, 'input_ids':input_ids_, 'offset': offset_ }
    
    

In [18]:
%%time

 train = (
    temp
    .drop('tokens_rec')
#     .fetch(10)
     .collect()
    .with_columns(
        pl.struct('token_rec').map_elements(lambda f: tokenize(f)).alias('ll')
    )
    .unnest('ll')
    .drop('token_rec')
    .explode(['txt', 'attention_mask', 'input_ids','offset' ])
    
)

CPU times: total: 11min 42s
Wall time: 21min 14s


In [19]:
# import joblib
train 

document,tokens,token_key,labels,txt,attention_mask,input_ids,offset
i64,list[str],list[i64],list[str],list[str],list[i64],list[i64],list[list[i64]]
7,"[""Design"", ""Thinking"", … "" ""]","[0, 0, … 752]","[""O"", ""O"", … ""O""]","[""design"", ""thinking"", … ""/""]","[1, 1, … 1]","[2640, 3241, … 1013]","[[0, 6], [7, 15], … [2171, 2172]]"
7,"[""Design"", ""Thinking"", … "" ""]","[0, 0, … 752]","[""O"", ""O"", … ""O""]","[""way"", "","", … ""[PAD]""]","[1, 1, … 0]","[2126, 1010, … 0]","[[2078, 2081], [2081, 2082], … [0, 0]]"
10,"[""Diego"", ""Estrada"", … "" ""]","[0, 0, … 562]","[""B-NAME_STUDENT"", ""I-NAME_STUDENT"", … ""O""]","[""diego"", ""estrada"", … ""on""]","[1, 1, … 1]","[5277, 26482, … 2006]","[[0, 5], [6, 13], … [2423, 2425]]"
10,"[""Diego"", ""Estrada"", … "" ""]","[0, 0, … 562]","[""B-NAME_STUDENT"", ""I-NAME_STUDENT"", … ""O""]","[""visual"", ""##ization"", … ""[PAD]""]","[1, 1, … 0]","[5107, 3989, … 0]","[[2310, 2316], [2316, 2323], … [0, 0]]"
16,"[""Reporting"", ""process"", … "" ""]","[0, 0, … 728]","[""O"", ""O"", … ""O""]","[""reporting"", ""process"", … ""the""]","[1, 1, … 1]","[7316, 2832, … 1996]","[[0, 9], [10, 17], … [2446, 2449]]"
…,…,…,…,…,…,…,…
22681,"[""Challenge"", "" "", … "" ""]","[0, 0, … 632]","[""O"", ""O"", … ""O""]","[""built"", ""suspense"", … ""[PAD]""]","[1, 1, … 0]","[2328, 23873, … 0]","[[2226, 2231], [2232, 2240], … [0, 0]]"
22684,"[""Brainstorming"", "" "", … "" ""]","[0, 0, … 678]","[""O"", ""O"", … ""O""]","[""brains"", ""##torm"", … ""takes""]","[1, 1, … 1]","[14332, 20654, … 3138]","[[0, 6], [6, 10], … [2361, 2366]]"
22684,"[""Brainstorming"", "" "", … "" ""]","[0, 0, … 678]","[""O"", ""O"", … ""O""]","[""also"", ""figured"", … ""[PAD]""]","[1, 1, … 0]","[2036, 6618, … 0]","[[2262, 2266], [2267, 2274], … [0, 0]]"


In [20]:
joblib.dump(train, 'train')

['train']